In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Dense
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output #Last FC layer's output  
    softmax_layer = Dense(256, activation='softmax')(vgg_out) #Create softmax layer taking input as vgg_ou
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False

    return tl_model
def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 8
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [2]:
label

256

In [3]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [4]:
np.shape(Y_train_label)

(2048, 256)

In [5]:
# #Output dim for your dataset
# output_dim = 256 #For Caltech256

# tl_model = getModel( output_dim )
# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
# for epoch in range(10):
#     start = 0
#     for batch in range(len(X_train)/64):
#         coll=io.ImageCollection(X_train[start:start+64])
#         images = convert_image(coll)
#         tl_model.train_on_batch(images, Y_train_label[start:start+64, :])
# #         tl_model.summary()
#         start += 64
# print 'stop here'

In [6]:
#Output dim for your dataset
output_dim = 256 #For Caltech256

#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [7]:
#Model
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error
# sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=True)
tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [9]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 1024 samples, validate on 1024 samples
Epoch 1/20
1024/1024 [==============================] - 20s - loss: 6.2708 - acc: 0.0518 - val_loss: 4.5489 - val_acc: 0.1631
Epoch 2/20
1024/1024 [==============================] - 22s - loss: 1.3294 - acc: 0.6934 - val_loss: 3.7921 - val_acc: 0.2695
Epoch 3/20
1024/1024 [==============================] - 41s - loss: 0.3817 - acc: 0.9463 - val_loss: 3.5783 - val_acc: 0.3037
Epoch 4/20
1024/1024 [==============================] - 42s - loss: 0.1496 - acc: 0.9922 - val_loss: 3.4869 - val_acc: 0.3203
Epoch 5/20
1024/1024 [==============================] - 42s - loss: 0.0706 - acc: 0.9971 - val_loss: 3.4270 - val_acc: 0.3301
Epoch 6/20
1024/1024 [==============================] - 42s - loss: 0.0363 - acc: 0.9990 - val_loss: 3.3775 - val_acc: 0.3418
Epoch 7/20
1024/1024 [==============================] - 41s - loss: 0.0200 - acc: 1.0000 - val_loss: 3.3344 - val_acc: 0.3496
Epoch 8/20
1024/1024 [==============================] - 42s - loss: 0.

In [11]:
print(hist.history)

{'acc': [0.0517578125, 0.693359375, 0.9462890625, 0.9921875, 0.9970703125, 0.9990234375, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'loss': [6.2707549184560776, 1.3294065129011869, 0.38165496755391359, 0.14961046539247036, 0.070614418713375926, 0.0362593510071747, 0.020049547427333891, 0.011828141752630472, 0.0073584155034041032, 0.0048052082420326769, 0.0032853434313437901, 0.0023464623955078423, 0.00174809681266197, 0.00135563442199782, 0.0010903109923674492, 0.00090493961943138856, 0.00077094227890484035, 0.00067083583962812554, 0.00059375638556957711, 0.00053279732310329564], 'val_acc': [0.1630859375, 0.26953125, 0.3037109375, 0.3203125, 0.330078125, 0.341796875, 0.349609375, 0.365234375, 0.37890625, 0.384765625, 0.384765625, 0.3876953125, 0.388671875, 0.3876953125, 0.3896484375, 0.396484375, 0.3984375, 0.3974609375, 0.3994140625, 0.3994140625], 'val_loss': [4.5489059090614319, 3.7920874133706093, 3.5782508254051208, 3.4869080409407616, 3.426998663693666

In [9]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto'),
    ModelCheckpoint(filepath="./weights.hdf5", verbose=1, save_best_only=True)
]

In [31]:
# tl_model.fit(X_train_data, Y_train_label,
#           nb_epoch=20,
#           batch_size=16)

tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=1, 
             verbose=1,  
             shuffle=None, class_weight=None, sample_weight=None)

Epoch 1/1
4096/4096 [==============================] - 40s - loss: 0.2756 - acc: 0.9353    


In [33]:
score = tl_model.evaluate(X_val_data, Y_val_label, batch_size=32, verbose=1, sample_weight=None)
print score

2695/2695 [==============================] - 26s    
[1.7824576142546418, 0.59962894253031884]


In [ ]:
import sys

# manual batch
nb_epoch = 20
nm_per_batch = 32
for e in range(nb_epoch):
    print 'Epoch', e
    batches = 0
    for X_batch, Y_batch in datagen.flow(X_train_data, Y_train_label, batch_size=nm_per_batch, shuffle=True):
        loss = tl_model.train_on_batch(X_batch, Y_batch)
#         print 'batches:', batches, ', loss:' , loss[0] , ', acc train:' , loss[1]
#         print 'batches:', batches, 'loss:' , loss[0],
        print 'batches:', batches,
        
        batches += 1
        if batches >= len(X_train) / nm_per_batch:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    
    #evaluate on validation
    score = tl_model.evaluate(X_val_data, Y_val_label, batch_size=nm_per_batch, verbose=1, sample_weight=None)
    print 'loss:' , loss[0] , ', acc train:' , loss[1], 'loss train: ', score